## 분산분석

- TSS(total sum of square): 종속변수값의 움직임 범위
$$\text{TSS} = \sum_{i=1}^N (y_i-\bar{y})^2 $$

- ESS(explained sum of squares): 회귀분석에 의해 예측된 y_hat의 분산
$$\text{ESS}=\sum_{i=1}^N (\hat{y}_i -\bar{\hat{y}})^2 $$

- RSS(residual sum of squares): 잔차의 분산
$$\text{RSS}=\sum_{i=1}^N (y_i - \hat{y}_i)^2\ = e^Te$$

$$\text{TSS} = \text{ESS} + \text{RSS}$$

- 모형 예측치의 움직임의 크기는 종속변수의 움직임의 크기보다 클 수 없다
- 모형의 성능이 좋을수록 모형 예측치의 움직임의 크기는 종속변수의 움직임의 크기와 비슷해진다.

In [6]:
from sklearn.datasets import make_regression
from statsmodels.api as sm

X0, y, coef = make_regression(
    n_samples=100, n_features=1, noise=30, coef=True, random_state=0)
dfX0 = pd.DataFrame(X0, columns=["X"])
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(y, columns=["Y"])
df = pd.concat([dfX, dfy], axis=1)

model = sm.OLS.from_formula("Y ~ X", data=df)
result = model.fit()

In [7]:
print("TSS = ", result.uncentered_tss)
print("ESS = ", result.mse_model)
print("RSS = ", result.ssr)
print("ESS + RSS = ", result.mse_model + result.ssr)
print("R squared = ", result.rsquared)

TSS =  291345.7578983061
ESS =  188589.61349210917
RSS =  102754.33755137534
ESS + RSS =  291343.9510434845
R squared =  0.6473091780922585


In [9]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     179.9
Date:                Tue, 06 Apr 2021   Prob (F-statistic):           6.60e-24
Time:                        05:00:24   Log-Likelihood:                -488.64
No. Observations:                 100   AIC:                             981.3
Df Residuals:                      98   BIC:                             986.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.4425      3.244     -0.753      0.4

In [10]:
sm.stats.anova_lm(result)

,df,sum_sq,mean_sq,F,PR(>F)
X,1.0,188589.613492,188589.613492,179.863766,6.601482e-24
Residual,98.0,102754.337551,1048.513648,NaN,NaN


In [13]:
# y와 y^의 샘플 상관계수 r의 제곱은 결정 계수 R2와 같다
np.corrcoef(y, result.fittedvalues)[0][1]**2

0.6473091780922592

In [14]:
from sklearn.datasets import load_boston

In [21]:
boston = load_boston()
X0 = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns = ['MEDV'])
X = sm.add_constant(X0)
df = pd.concat([X, y], axis = 1)

In [24]:
model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data = df)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Tue, 06 Apr 2021   Prob (F-statistic):          6.72e-135
Time:                        05:12:33   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.0

In [26]:
# F검정을 이용한 모형 비교
reduced_model = sm.OLS.from_formula("MEDV ~ CRIM + ZN + CHAS + NOX + RM + DIS + RAD + TAX + PTRATIO + B + LSTAT", data = df)
sm.stats.anova_lm(model.fit(), reduced_model.fit())

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,492.0,11078.784578,0.0,NaN,NaN,NaN
1,494.0,11081.363952,-2.0,-2.579374,0.057493,NaN


In [28]:
# F검정을 사용한 변수 중요도 비교
model_boston = sm.OLS.from_formula(
    "MEDV ~ CRIM + ZN + INDUS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT + CHAS", data=df)
result_boston = model_boston.fit()
sm.stats.anova_lm(result_boston, typ=2)

,sum_sq,df,F,PR(>F)
CRIM,243.219699,1.0,10.801193,1.086810e-03
ZN,257.492979,1.0,11.435058,7.781097e-04
INDUS,2.516668,1.0,0.111763,7.382881e-01
NOX,487.155674,1.0,21.634196,4.245644e-06
RM,1871.324082,1.0,83.104012,1.979441e-18
AGE,0.061834,1.0,0.002746,9.582293e-01
DIS,1232.412493,1.0,54.730457,6.013491e-13
RAD,479.153926,1.0,21.278844,5.070529e-06
TAX,242.257440,1.0,10.758460,1.111637e-03
PTRATIO,1194.233533,1.0,53.034960,1.308835e-12
